In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 689.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.5 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
url = "https://youtube138.p.rapidapi.com/search/"

querystring = {"q":"party 4 u","hl":"en","gl":"US"}

headers = {
	"x-rapidapi-key": "api key",
	"x-rapidapi-host": "youtube138.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'contents': [{'type': 'video', 'video': {'author': {'avatar': [{'height': 68, 'url': 'https://yt3.ggpht.com/HamyHLIfpmtOeEE9Myu3qdyRSnGi3iCWCm5KL7JJ403UG12z3MAVcWqDYRXQcwFrykI7GaC07ys=s88-c-k-c0x00ffffff-no-rj', 'width': 68}], 'badges': [], 'canonicalBaseUrl': None, 'channelId': 'UCOvEMf00DnwdkpOCRWdc6uA', 'title': 'Charli xcx'}, 'badges': ['4K'], 'descriptionSnippet': 'Director: Mitch Ryan Management: Sam Pringle, Twiggy Rowley, Brandon Creed Creative Director: Imogene Strauss Atlantic\xa0...', 'isLiveNow': False, 'lengthSeconds': 297, 'movingThumbnails': [{'height': 180, 'url': 'https://i.ytimg.com/an_webp/agu22bqGHto/mqdefault_6s.webp?du=3000&sqp=CMzSm8IG&rs=AOn4CLB2zEqdJkFP0gtk08B61LpVLVn7PQ', 'width': 320}], 'publishedTimeText': '3 weeks ago', 'stats': {'views': 11432261}, 'thumbnails': [{'height': 202, 'url': 'https://i.ytimg.com/vi/agu22bqGHto/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBS2cmGJ6v7ojW_OkJI7XftmLHx0w', 'width': 360}, {'height': 404, 'u

In [ ]:

@tool
def search_youtube_video(query: str, region: str = "US", language: str = "en") -> str:
    """
    Search for a video on YouTube using the RapidAPI YouTube138 API.

    Args:
        query (str): The search query (e.g., video title or keyword).
        region (str): The region code (default is 'US').
        language (str): The language code (default is 'en').

    Returns:
        str: The title and URL of the top video result.
    """
    url = "https://youtube138.p.rapidapi.com/search/"
    params = {
        "q": query,
        "hl": language,  # language
        "gl": region     # geographic location
    }
    headers = {
        "x-rapidapi-key": "api key",  # Replace with your RapidAPI key
        "x-rapidapi-host": "youtube138.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        # Extract the first video result
        for result in data.get("contents", []):
            video = result.get("video")
            if video:
                title = video.get("title")
                video_id = video.get("videoId")
                video_url = f"https://www.youtube.com/watch?v={video_id}"
                return f"Top result:\nTitle: {title}\nURL: {video_url}"

        return "No video found for your query."

    except requests.exceptions.RequestException as e:
        return f"Error during YouTube search: {str(e)}"
    except (KeyError, IndexError, TypeError) as e:
        return f"Unexpected response format: {str(e)}"

In [ ]:
youtube_assistant = create_react_agent(
    model=model,
    tools=[search_youtube_video],
    prompt="You are a helpful YouTube video assistant. Your job is to help users find YouTube videos based on their search queries.",
    name="youtube_assistant"
)

In [ ]:
response = youtube_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="5 minute video of yoga.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='5 minute video of yoga.', additional_kwargs={}, response_metadata={}, id='f85638f2-e133-48f4-b012-cbab3802f32c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pXsiy2GNEilA0LxDjtLj4AzW', 'function': {'arguments': '{"query":"5 minute yoga"}', 'name': 'search_youtube_video'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 168, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgYeCovVMictEICdOyoqYZMSixwFR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='youtube_assistant', id='run--42ed5e3a-74de-46a7-887d-46bf996a5f5b-0', tool_calls=[{'name': 'search_youtube_video', 'arg

In [ ]:
response["messages"][-1].content

'I found a video for you!\n\n**Title:** 5 Minute Yoga for Back Tension (6 Movements of the Spine)  \n**URL:** [Watch it here](https://www.youtube.com/watch?v=p39GdEUxxAk)'

# Class

In [26]:
import requests
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["RAPIDAPI_KEY"] = userdata.get("RAPIDAPI_KEY")
load_dotenv()

class YoutubeAgent:
    def __init__(self):
        self.openai_api_key = os.getenv('OPENAI_API_KEY')
        self.rapidapi_key = os.getenv('RAPIDAPI_KEY') # rapidapi_key or "api key here"  # Replace with your RapidAPI key
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini")
        self.agent = create_react_agent(
            model=self.model,
            verbose=True,
            tools=[self.search_youtube_video],
            prompt=(
    "You are a helpful YouTube video assistant. Your job is to help users find YouTube videos "
    "based on their search queries. Use the tool to perform a search and return the top result. "
    "Once the top result is found, return it as 'Final Answer: <result>' and stop."
),
            name="youtube_assistant"
        )
        # print(self.openai_api_key)
        # print(self.rapidapi_key)

    @tool
    def search_youtube_video(self, query: str, region: str = "US", language: str = "en") -> str:
        """
        Search for a video on YouTube using the RapidAPI YouTube138 API.
        Args:
            query (str): The search query (e.g., video title or keyword).
            region (str): The region code (default is 'US').
            language (str): The language code (default is 'en').
        Returns:
            str: The title and URL of the top video result.
        """
        print("*********************Tool Invoked")
        url = "https://youtube138.p.rapidapi.com/search/"
        params = {
            "q": query,
            "hl": language,
            "gl": region
        }
        headers = {
            "x-rapidapi-key": self.rapidapi_key,
            "x-rapidapi-host": "youtube138.p.rapidapi.com"
        }
        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            for result in data.get("contents", []):
                video = result.get("video")
                if video:
                    print("Hello")
                    title = video.get("title")
                    print("Hello", title)
                    video_id = video.get("videoId")
                    print("Hello", video_id)
                    video_url = f"https://www.youtube.com/watch?v={video_id}"
                    return f"Final Answer: Top result:\nTitle: {title}\nURL: {video_url}"
            return "Final Answer: No video found for your query."
        except requests.exceptions.RequestException as e:
            return f"Error during YouTube search: {str(e)}"
        except (KeyError, IndexError, TypeError) as e:
            return f"Unexpected response format: {str(e)}"

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        },
        config={"recursion_limit": 50})
        return response["messages"][-1].content


In [27]:
# Usage example for YoutubeAgent class
youtube_agent = YoutubeAgent()
result = youtube_agent.run("5 minute yoga video")
print(result)

TypeError: create_react_agent() got an unexpected keyword argument 'verbose'

In [20]:
# Usage example for YoutubeAgent class
youtube_agent = YoutubeAgent()
result = youtube_agent.run("5 minute yoga video")
print(result)

I'm having trouble retrieving the requested information at the moment. Please try again later.
